In [ ]:
import os
import numpy as np
from PIL import Image

def load_images_and_labels(folder_path):
    images = []
    labels = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):

            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            image = image.convert("RGB")
            
            image_array = np.array(image) / 255.0
            images.append(image_array)
            
            if ("target" in filename) & ("nontarget" not in filename):
                labels.append(1)
            elif "nontarget" in filename:
                labels.append(-1)
            else:
                raise ValueError("Invalid filename format")
    
    return np.array(images), np.array(labels)

In [ ]:
folder_path = "/Users/yohanabeysinghe/Mac/1. ENTC/FYP - Ear EEG/Datasets/ConvertedToImages"
X, Y = load_images_and_labels(folder_path)

print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape) 

In [ ]:
import matplotlib.pyplot as plt

# Select 10 random indices
num_samples = X.shape[0]
random_indices = np.random.choice(num_samples, size=10, replace=False)

# Plot the selected images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, idx in enumerate(random_indices):
    row = i // 5
    col = i % 5
    axes[row, col].imshow(X[idx])
    axes[row, col].set_title("Label: {}".format(Y[idx]))
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
print(Y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

num_samples, height, width, channels = X.shape
X_flat = X.reshape(num_samples, height * width * channels)

X_train, X_test, Y_train, Y_test = train_test_split(X_flat, Y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, Y_train)
Y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)


In [ ]:
X[0].shape

In [ ]:
from lime import lime_image

explainer = lime_image.LimeImageExplainer()

image_to_explain = X_test[0]
label_to_explain = Y_test[0]

explanation = explainer.explain_instance(image_to_explain, rf_classifier.predict_proba, top_labels=2, hide_color=0, num_samples=1000)

temp, mask = explanation.get_image_and_mask(label_to_explain, positive_only=True, num_features=5, hide_rest=False)
plt.imshow(lime_image.mark_boundaries(temp / 2 + 0.5, mask))
plt.title("Explanation for class {}".format(label_to_explain))
plt.axis('off')
plt.show()